# 1. Setup

Seção para realizar a prepração do ambiente de desenvolvimento, realizando a conexão com o Google Drive e importando as bibliotecas necessárias e importação das bases fornecidas, transformando todas em uma só.


In [7]:
# Montagem do drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Download Spacy em português
!python -m spacy download pt_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.7.0/pt_core_news_sm-3.7.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
# Importar bibliotecas necessárias
import pandas as pd
import spacy
import re

In [10]:
# Carregamento das informações do SPACY
nlp = spacy.load("pt_core_news_sm")

## Carregamento dos dados

Para realizar o pré processamento foi utilizada 3 bases de dados que possuem semelhanças:

Base 1: Base original apenas com conversas e intenções

Base 2: Base secundária com conversas e mais de uma interação

Base 3: Base terciária com um dicionário para as palavras em japonês

In [11]:
# Alterar caminhos para onde estão armazenadas a bases
PATH = '/content/base1_v1_20240904.csv'
PATH2 = '/content/base2_v1_20240904.xlsx'
PATH3 = '/content/base3_v1_20240904.xlsx'

base = pd.read_csv(PATH, sep=';')
base2 = pd.read_excel(PATH2, sheet_name='Q&A')
base3 = pd.read_excel(PATH3)

### Tratamento básico das bases

Retirar linhas duplicadas, filtrar colunas principais e exploração da base.

In [12]:
# Filtrando apenas colunas importantes retirando os Index
base = base[['Intencao','Pergunta','Resposta\n']]
base2 = base2[['Intencao', 'Pergunta','Resposta\n']]
base3 = base3[['Intencao', 'Pergunta','Resposta\n']]

In [13]:
print('Tamanho database 1: ', len(base))
print('Tamanho database 2: ', len(base2))
print('Tamanho database 3: ', len(base3))

Tamanho database 1:  981
Tamanho database 2:  999
Tamanho database 3:  728


In [14]:
# Junção das bases de dados
baseGeral = pd.merge(base, base2, how='outer')
baseGeral = pd.merge(baseGeral, base3, how='outer')

In [15]:
# Renomear colunas
baseGeral.rename(columns={'Resposta\n':'Resposta'}, inplace=True)

In [16]:
# Retirar linhas que estão duplicadas
baseGeral.drop_duplicates(inplace=True)

In [17]:
# Visualização
baseGeral.head()

,Intencao,Pergunta,Resposta
0,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d..."
1,Como fazer remessa,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f..."
2,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ..."
152,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...
153,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...


In [18]:
len(baseGeral)

799

# 2. Pipeline

**Etapa importante para lidar com os textos do dataframe futuramente.**

**Nesta etapa, o processo de pré-processamento de dados foi separado nas seguintes etapas a seguir:**

Limpeza dos dados do dataframe: remover os espaços em excesso, converter o texto em uma frase de uma linha, normalizar o texto para que fique todo com letras minúsculas, remover as pontuações, números, símbolos, caracteres especiais e emojis, links e endereços de e-mail;

## Normalização e remoção de dados que não serão utilizados

In [19]:
# Dar espaços após as pontuações
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'([.,!?;:])', r'\1 ', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'([.,!?;:])', r'\1 ', str(x)))

# Substituir quebras de linha por espaços
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\n', ' ', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\n', ' ', str(x)))

# Normalizar o dataframe para letras minúsculas
baseGeral['Pergunta'] = baseGeral['Pergunta'].str.lower()
baseGeral['Resposta'] = baseGeral['Resposta'].str.lower()

# Remover pontuações do dataframe
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

# Remover números do dataframe
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\d+', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\d+', '', str(x)))

# Remover espaços extras do dataframe
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))

# Remover URLs
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'http\S+', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

# Remover emails
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\S+@\S+', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\S+@\S+', '', str(x)))

# Remover emojis
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)))

# Remover palavras que contém apenas um caractere como aaa, bbbb
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\b(\w)\1+\b', '', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\b(\w)\1+\b', '', str(x)))


## Remoção de stopwords

Remove as "palavras de parada", que são os artigos, pronomes e palavras usadas com muita frequência e acabam tendo pouco significado no texto e, em alguns casos, se tornando irrelevantes;

In [20]:
# Função para remover stopwords
def remove_stopwords(texto):
    doc = nlp(texto)
    # Retornar palavras que não são stopwords
    return " ".join([token.text for token in doc if not token.is_stop])

In [21]:
# Aplicar a função em cada linha da coluna de texto
baseGeral['Pergunta_StopWord'] = baseGeral['Pergunta'].apply(remove_stopwords)
baseGeral['Resposta_StopWord'] = baseGeral['Resposta'].apply(remove_stopwords)

## Dicionário das abreviações encontradas na base de dados

Dicionário para conversão de abreviações para palavras por extenso: foi feita uma análise das principais abreviações que apareciam no dataframe e, com base nisso, criado um dicionário que é usado para substituir cada abreviação por sua palavra por extenso através de um loop que percorre as principais colunas que estamos usando;


In [22]:
# Criação de um dicionário para tratar abreviações do dataframe
abreviacoes = {
    'add': 'adicionar',
    'app': 'aplicativo',
    'pq': 'porque',
    'vc': 'você',
    'vcs': 'vocês',
    'sr': 'senhor',
    'sra': 'senhora',
    'srto': 'senhorito',
    'srta': 'senhorita',
    'qdo': 'quando',
    'qdo': 'quando',
    'qt': 'quanto',
    'qto': 'quanto',
    'pra': 'para',
    'q': 'que',
    'p': 'para',
    'pf': 'por favor',
    'yenes': 'ienes',
    'yens': 'ienes',
    'iens': 'ienes',
    'hj': 'hoje',
    'obg': 'obrigado',
    'td': 'tudo',
    'bm': 'bom',
}

In [23]:
# Substituindo as abreviações das palavras para extenso em toda a base de dados de uma vez
def substituir_abreviacoes(texto, abreviacoes):
    for abreviacao, palavra in abreviacoes.items():
        texto = texto.str.replace(r'\b' + abreviacao + r'\b', palavra, regex=True)
    return texto

# Aplicando a função para substituir abreviações nas colunas específicas
baseGeral['Pergunta_Abrev'] = substituir_abreviacoes(baseGeral['Pergunta_StopWord'], abreviacoes)
baseGeral['Resposta_Abrev'] = substituir_abreviacoes(baseGeral['Resposta_StopWord'], abreviacoes)

In [24]:
baseGeral.head(1)

,Intencao,Pergunta,Resposta,Pergunta_StopWord,Resposta_StopWord,Pergunta_Abrev,Resposta_Abrev
0,Como depositar,boa dia tudo bem eu gostaria de saber sobre aq...,bom dia sim o sr pode utilizar o carto de deps...,dia gostaria caixa family mart verde eh horas ...,dia sr utilizar carto depsitos mquinas horas e...,dia gostaria caixa family mart verde eh horas ...,dia senhor utilizar carto depsitos mquinas hor...


## Lematização dos dados

Lematização: o processo de lematização identifica qual é a palavra e a transforma em sua forma base ou lema, o que é útil para normalização e interpretação do texto

In [25]:
# Função para lematizar o texto
def lematizar_texto(texto):
    doc = nlp(texto)
    # Juntar os lemas das palavras em uma string
    return " ".join([token.lemma_ for token in doc])

# Aplicar a lematização nas colunas 'Pergunta_Abrev' e 'Resposta_Abrev'
baseGeral['Pergunta_Lematizada'] = baseGeral['Pergunta_Abrev'].apply(lematizar_texto)
baseGeral['Resposta_Lematizada'] = baseGeral['Resposta_Abrev'].apply(lematizar_texto)

In [31]:
# Apagar colunas não utilizadas e eliminar linhas com valores nulos
baseGeral = baseGeral.dropna()
baseGeral = baseGeral.drop_duplicates()
baseGeral.info()

<class 'pandas.core.frame.DataFrame'>
Index: 782 entries, 0 to 941932
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Intencao             782 non-null    object
 1   Pergunta             782 non-null    object
 2   Resposta             782 non-null    object
 3   Pergunta_StopWord    782 non-null    object
 4   Resposta_StopWord    782 non-null    object
 5   Pergunta_Abrev       782 non-null    object
 6   Resposta_Abrev       782 non-null    object
 7   Pergunta_Lematizada  782 non-null    object
 8   Resposta_Lematizada  782 non-null    object
dtypes: object(9)
memory usage: 61.1+ KB


In [32]:
# Salvar a base de dados tratada
baseGeral['Intencao'].value_counts()/len(baseGeral)

,count
Intencao,
Confirmacao de cambio/taxas,0.152174
"Pedido de envio via metodo ""ByPhone""",0.127877
Problemas/Duvidas de atualizacao de dados cadastrais,0.102302
Problemas/Duvidas sobre remessas,0.083120
Como se inscrever,0.075448
Problemas/Duvidas sobre deposito,0.061381
Tempo de remessa,0.052430
Solicitacao de cartao de remessas,0.044757
Termos e condicoes do servico,0.042199


## Baixar base tratada

In [34]:
baseGeral.to_excel('baseTratada.xlsx')